## Feature Selection

#### OBJECTIVE: Identify good feature subsets using the training dataset and test these subsets on the test data.

## Case I: NONE Feature Selection
Here,there's no feature selection strategy is incorporated. 

All the features are considered while calculating the accuracy on test data as well on train data using hold out and cross validation respectively.

In [ ]:
#Importing necessary libraries.

import pandas as pd                                         
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

In [ ]:
#Training data and Test data is read into dataframes train_data and test_data

train_data = pd.read_csv('SPECTF_train.csv')
test_data =  pd.read_csv('SPECTF_test.csv') 

In [ ]:
# The target variable DIAGNOSIS is popped from the train as well as test data and stored in y_train and y_test variables.

y_train = train_data.pop('DIAGNOSIS')
y_test =  test_data.pop('DIAGNOSIS')

In [ ]:
# The data from train_data and test_data except the DIAGNOSIS column is saved into the X_train and X_test variables respectively.

X_train = train_data
X_test =  test_data

In [ ]:
# Printing the shape of both X_train and X_test data just to ensure they have same number of columns.

print(X_train.shape)
print(X_test.shape)

Gradient boosting is a machine learning technique for regression and classification issues 
that produces a predictive model in the form of a collection of weak predictive models. It builds the model in a stage-wise fashion and generalizes it by allowing optimization of an arbitrary differentiable loss function.

The ensemble problem is simplified greedily as a forward stage-wise additive model. 
We don’t optimize the ensemble in a global manner, but instead seek to improve the result based on the current estimate.

In [ ]:
# Gradient Boosting classifier is built using the GradientBoostingClassifier library from sklearn.ensemble module.
# loss is set to deviance for a binary classification (DIAGNOSIS 1/0).
# Learning_rate (default 0.1)
# random_state is set to 1 so as to reproduce the same output every time it is executed.

gbc = GradientBoostingClassifier(loss = 'deviance',learning_rate=0.1, random_state=0)

In [ ]:
# Gradient boosting classifier is used to learn the model parameters using the fit() method.

gbc.fit(X_train,y_train)

## Test on Test Data using Hold Out method

In [ ]:
# The output class(DIAGNOSIS) is predicted for the X_test values using the predict().

y_pred_test=gbc.predict(X_test)

# Accuracy of predicted values (y_pred_test) with actual values (y_test) is calculated using the accuracy_score() method

acc_holdout = accuracy_score(y_pred_test,y_test)
print ("Accuracy with Hold Out method when None features extracted:", acc_holdout)

## Test on Train Data using Cross Validation method

In [ ]:
# For 5 folds (1 Test data and rest 4 are treated as training data), accuracies are computed using the method cross_val_Score()
# Here,Cross validation is done on the training data only (Internally it considers 1 part as test and rest as training data)
# The classifier it uses to classify the records with DIAGNOSIS as 1 or 0 is GradientBoostingClassifier.
scores = cross_val_score(gbc, X_train, y_train, cv=5)

# Accumulated accuracy of all the whole training data is computed by taking the mean of the scores
acc_cross_val = scores.mean()
print ("Accuracy with Cross-validation method when None features extracted:", acc_cross_val)

In [ ]:
df_None = pd.DataFrame([[acc_holdout,acc_cross_val]],index=['None'],columns=['Test','Train'])
print(df_None)
df_None.plot(kind = 'bar',width = 0.3,color= ['red','green'])
plt.title('Fig1. Bar graph showing accuracies of both Train and Test Data when None features extracted')
plt.xlabel('Feature Selection Strategy: None')
plt.ylabel('Accuracy')
plt.legend(loc="upper right")
plt.show()

Conclusion from Case I: Accuracy generated by K-fold cross validation is higher than that done by hold-out methos on Test data. It is because Cross validation method is more precise but slightly more computationally expensive as it considers every point of the input data as test data once and rest of the times as training data .

## Case II: Feature Selection using INFORMATION GAIN

Information gain (IG) measures how much “information” a feature gives us about the target class.

Features out of 44 which are having high mutual dependency with the target variable are extracted first.

Then instead of training the model on whole dataset, model is build using only the extracted features.

Then classification of unknown values with only extracted features is done.

Thus computation complexity reduces as compared to training the model with all the features.

In case of a classification task, we measure the dependence between each feature and the target variable. 
If there exists a dependence, then we extract that feature for our classification tasks. 

In [ ]:
# Mutual dependency between 2 random variables (All the features and target variables) is computed  using the mutual_info_classif()
# of feature.selection module.
# For every feature, the dependency score is stored in a dictionary.

mi = dict()
i_scores = mutual_info_classif(X_train, y_train)

for i,j in zip(train_data.columns,i_scores):
    mi[i]=j

# A dataframe from dictionary of mutual dependency score is built and sorted on the basis of i_scores in descending order
 
df_IGain = pd.DataFrame.from_dict(mi,orient='index',columns=['I-Gain'])
df_IGain.sort_values(by=['I-Gain'],ascending=False,inplace=True)
df_IGain.head(10)

In [ ]:
# A plot of all the features along with their dependency score(I-Gain score) is plotted

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize = (12,15))
n = len(df_IGain.index)
rr = range(1,n)
fig, ax = plt.subplots()
ax.bar(df_IGain.index, df_IGain["I-Gain"], label='I-Gain',width=.80)
ax.set_xticklabels(list(df_IGain.index), rotation = 90)
ax.set_xlabel('Features')
ax.set_ylabel('I-Gain')
ax.legend()

plt.show()

## Select *k* best features

We rank the features using information gain (well mutual information) and select the _k_ best to build a classifier.  
We iterate through increasing values of *k*.  
`SelectKBest` is a _transform_ that transforms the training data.

The k best features out of 44 which are having high accuracy from the classifier passed(mutual_info_classif) are extracted. 

Training and test data is transformed according to the number of features extracted.

Then it learns the model parameters.

(Hold out Method): Prediction for the class of test data is done using the trained classifier.

(Cross Val Method): Cross validation is performed on the training data to calculate its accuracy.

In [ ]:
# Out of 44 features,k features are extracted and model parameters are found on X_train and y_train.
# Both train and test data are transformed(only k features from both are selected.)
# Accuracy is found while predicting the class of transformed X_test with the actual values of target.
# For every feature based on mutual_info_classif, accuracy is stored in a dataframe.

acc_scores = []
for kk in range(1, X_train.shape[1]+1):
    FS_trans = SelectKBest(mutual_info_classif,k=kk).fit(X_train, y_train)
    X_tR_new = FS_trans.transform(X_train)
    X_tS_new = FS_trans.transform(X_test)
    
    gbc_Igain=gbc.fit(X_tR_new, y_train)
    y_pred_Igain = gbc_Igain.predict(X_tS_new)
    acc = accuracy_score(y_test, y_pred_Igain)
    acc_scores.append(acc)

df_IGain['Accuracy'] = acc_scores
df_IGain.head(5)

In [ ]:
# A plot with all the features along with their I-Gain score is plotted.
# Accuracy values for all the features is plotted (Accuracy at point x comprises the accuracy when all the features preceding x are considered.)

import matplotlib.pyplot as plt
#%matplotlib inline

plt.figure(figsize = (20,20),dpi= 150)
n = len(df_IGain.index)
rr = range(1,n)
fig, ax = plt.subplots()
ax2 = ax.twinx()
ax.bar(df_IGain.index, df_IGain["I-Gain"], label='I-Gain',width=.35)
ax2.plot(df_IGain.index, df_IGain["Accuracy"], color='red', label='Accuracy')
ax.set_xticklabels(list(df_IGain.index), rotation = 90)
ax.set_xlabel('Features')
ax.set_ylabel('I-Gain')
ax2.set_ylabel('Accuracy')
ax.legend()

plt.show()

In [ ]:
# A list of k selected features is extracted from the dataframe with accuracy value <= index of first occuence of maximum.
# idxmax() function returns index of first occurrence of maximum over requested axis.

selected_features_list = list(df_IGain[:df_IGain['Accuracy'].idxmax()].index.values)
print(selected_features_list)

## Test on Test Data using Hold Out method

In [ ]:
# Now the train data comprises of only the selected features out of 44 features.
X_train_Igain = X_train[selected_features_list]

gbc.fit(X_train_Igain,y_train)

# The output class(DIAGNOSIS) is predicted for the X_test values using the predict().
y_pred_test_Igain=gbc.predict(X_test[selected_features_list])

# Accuracy of predicted values (y_pred_test_Igain) with actual values (y_test) is calculated using the accuracy_score() method
acc_holdout_igain = accuracy_score(y_pred_test_Igain,y_test)
print ("Accuracy with Hold Out method when k =", len(selected_features_list),"best features extracted:", acc_holdout_igain)

## Test on Train Data using Cross Validation method

In [ ]:
# For 5 folds (1 Test data and rest 4 are treated as training data), accuracies are computed using the method cross_val_Score()
# Here,Cross validation is done on the training data only (Internally it considers 1 part as test and rest as training data)
# The classifier it uses to classify the records with DIAGNOSIS as 1 or 0 is GradientBoostingClassifier.

scores = cross_val_score(gbc, X_train_Igain, y_train, cv=5)

# Accumulated accuracy of all the whole training data is computed by taking the mean of the scores
acc_cross_val_igain = scores.mean()
print ("Accuracy with Cross-validation method when k =", len(selected_features_list),"best features extracted:", acc_cross_val_igain)

In [ ]:
df_igain = pd.DataFrame([[acc_holdout_igain,acc_cross_val_igain]],index=['Igain'],columns=['Test','Train'])
print(df_igain)
df_igain.plot(kind = 'bar',width = 0.3,color= ['red','green'])
plt.title('Fig2. Bar graph showing accuracies of both Train and Test Data when k features extracted using I-Gain value')
plt.xlabel('Feature Selection Strategy: I-Gain')
plt.ylabel('Accuracy')
plt.legend(loc="upper right")
plt.show()

## Case III: Feature Selection using Wrapper-based forward sequential search

scikit learn does not support Wrapper feature selection so we use MLxtend.

Wrapper methods are based on greedy search algorithms as they evaluate all possible combinations of the features
and select the combination that produces the best result for a specific machine learning algorithm.

Step Forward Feature Selection: In the first phase,the performance of the classifier is evaluated corressponding to each feature. The feature that performs the best is selected out of all the features.

Next step, the first feature is tried in combination with all the other features. The combination of two features that yield the best algorithm performance is selected. 

Similarly, the process continues until the specified number of features are selected.

k_features specifies the number of features to select.
The forward parameter, if set to True, performs step forward feature selection. 
The verbose parameter is used for logging the progress of the feature selector.
The scoring parameter defines the performance evaluation criteria.
Finally, cv refers to cross-validation folds.

Thus a feature selector is created using the SFS().
Now the fit method is invoked on the feature selector and pass it the training and test sets.

In [ ]:
# SequentialFeatureSelector is built with estimator as Gradient Boosting Classifier and folds 10 using which model parameters are made using the fit().
feature_names = train_data.columns

sfs_forward = SFS(gbc, 
                  k_features=44, 
                  forward=True, 
                  floating=False, 
                  verbose=1,
                  scoring='accuracy',
                  cv=10, n_jobs = -1)

sfs_forward = sfs_forward.fit(X_train, y_train,custom_feature_names=feature_names)

In [ ]:
# A plot showing all the features along with their accuracy is shown.
# sfs_forward.get_metric_dict() stores pair of(feature and its corresponding standard deviation, cross validation accuracies, average accuracy and so on)

fig1 = plot_sfs(sfs_forward.get_metric_dict(),ylabel='Accuracy',kind='std_dev', figsize = (8,5))
plt.ylim([0.5, 1])
plt.title('Sequential Forward Selection (w. StdDev)')
plt.grid()
plt.show()
#print(sfs_forward.k_feature_names_)

In [ ]:
df_res = pd.DataFrame(pd.DataFrame(sfs_forward.get_metric_dict()).loc['avg_score'].astype('float64'))
num_features = df_res.idxmax()
num = int(num_features[0])
sfs_forward.subsets_[num]['feature_names']

In [ ]:
# Maximum accuracy comes out to be at feature number 7, so k_feature is set to 7 here.
# To create forward based selection strategy, the attribute forward is set to True.

feature_names = train_data.columns
sfs_forward = SFS(gbc, 
                  k_features=num, 
                  forward=True, 
                  floating=False, 
                  verbose=1,
                  scoring='accuracy',
                  cv=10, n_jobs = -1)

#print(sfs_forward)
sfs_forward = sfs_forward.fit(X_train, y_train,custom_feature_names=feature_names)

In [ ]:
# A plot showing 7 features from the dictionary (sfs_forward.get_metric_dict()) along with their accuracy is shown.
# sfs_forward.get_metric_dict() stores pair of (feature and its corresponding standard deviation, cross validation accuracies, average accuracy and so on)

fig1 = plot_sfs(sfs_forward.get_metric_dict(),ylabel='Accuracy',kind='std_dev')
plt.ylim([0.5, 1])
plt.title('Sequential Forward Selection (w. StdDev)')
plt.grid()
plt.show()
print(sfs_forward.k_feature_names_)

In [ ]:
# All the selected combinations of features is shown along with their average_score, feature names and so on are shown using the subsets_ function of SFS.
sfs_forward.subsets_

## Test on Test Data using Hold Out method

In [ ]:
# Now the train data comprises of only the selected 7 features out of 44 features.

selected_features_train = X_train[list(sfs_forward.k_feature_names_)]
selected_features_test = X_test[list(sfs_forward.k_feature_names_)]

gbc.fit(selected_features_train,y_train)

# The output class (DIAGNOSIS) is predicted for the selected features from the X_test values using the predict().
y_pred_test_forward=gbc.predict(selected_features_test)

# Accuracy of predicted values (y_pred_test_forward) with actual values (y_test) is calculated using the accuracy_score() method

acc_holdout_forward = accuracy_score(y_pred_test_forward,y_test)
print ("Accuracy with Hold Out method when k =", len(selected_features_list),"best features extracted using SFS:", acc_holdout_forward)

## Test on Train Data using Cross Validation method

In [ ]:
# As the SFS internally uses cross validation on training data with 10 folds. So k_score_ function is used to find the accuracy of training data when done cross validation on it.
# Out of 10, 1 part is considered as test data and rest treated as trainig data.
# The classifier it uses to classify the records with DIAGNOSIS as 1 or 0 is GradientBoostingClassifier.
# Accumulated accuracy of all the whole training data is computed by taking the mean of the scores.

print ("Accuracy with Cross-validation method when k =", len(selected_features_list),"best features extracted using SFS:", sfs_forward.k_score_)

In [ ]:
# A dataframe with columns as Test and Train and index as SFS and values of accuracy with hold out and accuracy with cross val is made
# A bar graph showing both the accuracies is plot.
df_frwd = pd.DataFrame([[acc_holdout_forward,sfs_forward.k_score_]],index=['SFS'],columns=['Test','Train'])
print(df_frwd)
df_frwd.plot(kind = 'bar',width = 0.3,color= ['red','green'])
plt.title('Fig3. Bar graph showing accuracies of both Train and Test Data when k features extracted using SFS')
plt.xlabel('Feature Selection Strategy: Wrapper Based SFS')
plt.ylabel('Accuracy')
plt.legend(loc="upper right")
plt.show()

## Case IV: Feature Selection using Wrapper-based backward elimination.

scikit learn does not support Wrapper feature selection so we use MLxtend.

Wrapper based backward elimination is a step backwards feature selection. 
It is the exact opposite of step forward feature selection.
In the first phase, one feature is removed in round-robin fashion from the feature set and the performance of the classifier is evaluated.

The feature set that yields the best performance is retained. 

In the second step, again one feature is removed in a round-robin fashion and the performance of all the combination of features except the 2 features is evaluated. 

This process continues until the specified number of features remain in the dataset.

k_features specifies the number of features to select.
The forward parameter, if set to False, performs step backward feature selection. 
The verbose parameter is used for logging the progress of the feature selector.
The scoring parameter defines the performance evaluation criteria.
Finally, cv refers to cross-validation folds.

Thus a feature selector is created using the SFS(forward = False).
Now the fit method is invoked on the feature selector and pass it the training and test sets.

In [ ]:
# SequentialFeatureSelector is built with estimator as Gradient Boosting Classifier and folds 10 but in backward direction using which model parameters are made using the fit().

feature_names = train_data.columns

sfs_backward = SFS(gbc, 
                  k_features=1, 
                  forward=False, 
                  floating=False, 
                  verbose=1,
                  scoring='accuracy',
                  cv=10, n_jobs = -1)

sfs_backward = sfs_backward.fit(X_train, y_train,custom_feature_names=feature_names)

In [ ]:
# A plot showing all the features along with their accuracy is shown.
# sfs_backward.get_metric_dict() stores pair of(feature and its corresponding standard deviation, cross validation accuracies, average accuracy and so on)

fig2 = plot_sfs(sfs_backward.get_metric_dict(),ylabel='Accuracy',kind='std_dev')
plt.ylim([0.5, 1])
plt.title('Sequential Backward Selection (w. StdDev)')
plt.grid()
plt.gca().invert_xaxis()
plt.show()
#print(sfs_backward.k_feature_names_)

In [ ]:
sfs_backward.get_metric_dict()

In [ ]:
df_res = pd.DataFrame(pd.DataFrame(sfs_backward.get_metric_dict()).loc['avg_score'].astype('float64'))
num_features = df_res[::-1].idxmax()
num = int(num_features[0])
sfs_backward.subsets_[num]['feature_names']

In [ ]:
# Maximum accuracy comes out to be at feature number 16, so k_feature is set to 7 here.
# To create backward based selection strategy, the attribute forward is set to False.

feature_names = train_data.columns

sfs_backward = SFS(gbc, 
                  k_features=num, 
                  forward=False, 
                  floating=False, 
                  verbose=1,
                  scoring='accuracy',
                  cv=10, n_jobs = -1)

sfs_backward = sfs_backward.fit(X_train, y_train,custom_feature_names=feature_names)

In [ ]:
# All the selected combinations of features is shown along with their average_score, feature names and so on are shown using the subsets_ function of SFS.
sfs_backward.subsets_

In [ ]:
fig1 = plot_sfs(sfs_backward.get_metric_dict(),ylabel='Accuracy',kind='std_dev')
plt.ylim([0.5, 1])
plt.title('Sequential Backward Elimination')
plt.grid()
plt.gca().invert_xaxis()
plt.show()
print(sfs_backward.k_feature_names_)

## Test on Test Data using Hold Out method

In [ ]:
# Now the train data comprises of only the selected 7 features out of 44 features.

selected_features_train = X_train[list(sfs_backward.k_feature_names_)]
selected_features_test = X_test[list(sfs_backward.k_feature_names_)]


gbc.fit(selected_features_train,y_train)

# The output class (DIAGNOSIS) is predicted for the selected features from the X_test values using the predict().
y_pred_test_backward=gbc.predict(selected_features_test)

# Accuracy of predicted values (y_pred_test_backward) with actual values (y_test) is calculated using the accuracy_score() method
acc_holdout_backward = accuracy_score(y_pred_test_backward,y_test)
print ("Accuracy with Hold Out method when k =", len(selected_features_list),"best features extracted using SFS backward:", acc_holdout_backward)

## Test on Train Data using Cross Validation method

In [ ]:
# As the SFS internally uses cross validation on training data with 10 folds. So k_score_ function is used to find the accuracy of training data when done cross validation on it.
# Out of 10, 1 part is considered as test data and rest treated as trainig data.
# The classifier it uses to classify the records with DIAGNOSIS as 1 or 0 is GradientBoostingClassifier.
# Accumulated accuracy of all the whole training data is computed by taking the mean of the scores.

print ("Accuracy with Cross-validation method when k =", len(selected_features_list),"best features extracted using SFS backward :", sfs_backward.k_score_)

In [ ]:
# A dataframe with columns as Test and Train and index as SFS_bkwd and values of accuracy with hold out and accuracy with cross val is made
# A bar graph showing both the accuracies is plot.

df_bkwd = pd.DataFrame([[acc_holdout_backward,sfs_backward.k_score_]],index=['SFS_Backward'],columns=['Test','Train'])
print(df_bkwd)
df_bkwd.plot(kind = 'bar',width = 0.3,color= ['red','green'])
plt.title('Fig4. Bar graph showing accuracies of both Train and Test Data when k features extracted using SFS Backward')
plt.xlabel('Feature Selection Strategy: Wrapper Based Backward Elimination')
plt.ylabel('Accuracy')
plt.legend(loc="upper right")
plt.show()

### CUMULATIVE REPRESENTATION OF ALL FEATURE SELECTION STRATEGY

A dataframe is built with columns as Test and Train and index having all 4 strategy of feature selection (None, I-Gain, SFS Forward, SFS Backward) and values of accuracy with hold out and cross validation.

In [ ]:
df = pd.DataFrame([[acc_cross_val,acc_holdout],[acc_cross_val_igain,acc_holdout_igain],[sfs_forward.k_score_,acc_holdout_forward],[sfs_backward.k_score_,acc_holdout_backward]],
                  index = ['None','I-Gain','Wrapper-F','Wrapper-B'],columns = ['Train','Test'])
df

In [ ]:
# A bar graph showing both the accuracies is plot.

df.plot(kind='bar',legend= 'upper right')
plt.title('Fig5. Bar graph showing accuracies of Hold Out and Cross Validation of all 4 Feature Selection Strategies.')
plt.xlabel('Feature Selection Strategy')
plt.ylabel('Accuracy')

### CONCLUSION

1. FILTER METHOD: The Information Gain filter is classifier agnostic pre-selection method which are independent of the machine learning method. This is computationally more efficient but selection of features may depend on the learning algorithm which it ignores.

2. WRAPPER METHOD: Wrappers are feedback methods which incorporate the Machine Learning algorithm in the Feature Selection process,i.e they rely on the performance of a specific classifier to evaluate the quality of a set of features. 
Wrapper methods search through the space of feature subsets and calculate the estimated accuracy of a single learning algorithm for each feature that can be added to or removed from the feature subset. 

The accuracy on the test data calculated using all 3 feature selection strategies (Information Gain Filter, Wrapper Sequential Forward Selection, Wrapper based Backward elimination) comes to be consistent whereas accuracy on the training dataset is eventually increasing in case of Wrapper methods.

The filter method is unstable as compared to the Wrapper based methods because:
1. No Model Bias:
Different features may suit different learning algorithms but filters do not take this into account.

2. No Feature Dependencies:
• The features are considered in isolation from one another, and are not considered in context.
• In some cases, a filter might select two predictive but correlated features, where one would be sufficient.
• In other cases, one feature needs another feature to boost accuracy, but a filter cannot discover this.

whereas the Wrapper based methods considers bias into account and considers the features in context.
The wrapper method is computationally more demanding, but takes dependencies of the feature subset on the learning algorithm into account.

Wrapper based Sequential Forward Selection method is faster than backward elimination technique as it starts with small subsets, so requires less running time if stopped early.
1. It starts from 1 feature and goes upto the maximum accuracy bearing features but it's accuracy is less than that of the backward elimination strategy as once it gets the maximum accuracy in between, it will stop considering the next features 
2. The Wrapper based Backward Elimination strategy is slow but accuracy is high as it starts from combination of all the features calculating the accuracy and removing the feature one by one till no feature left.

The discrepancy between the training and test data can be accompanied by two facts:
1. The training data set size is not even the half of the test dataset which may lead to overfitting.
2. Using features extracted from wrapper methods can lead to overfitting as wrapper methods already train machine learning models with the features and it affects the true power of learning. But the features from filter methods will not lead to overfitting in most of the cases.

From the result graph, the accuracy on the test dataset is similar throughout the feature selection strategy but the accuracy on the training dataset in eventually increasing in the case of Wrapper based methods.

The insights on data that can be derived from the overall analysis of the data:
    1. Overfitting may happen due to the insufficient training data set.
    2. Filter methods may fail to find the best subset of features in situations when there is not enough data to model the statistical correlation of the features, but wrapper methods can always provide the best subset of features because of their exhaustive nature.